In [ ]:
!pip install -q transformers==4.41.2 datasets==2.19.1 sentencepiece==0.1.99 evaluate==0.4.1 sacrebleu==2.3.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 78.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 136.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.3.

In [ ]:
!pip install -U transformers==4.45.1 accelerate==0.34.2 datasets==3.0.2 sentencepiece==0.2.0 evaluate==0.4.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Output.zip to Output.zip


In [ ]:
import zipfile
import io

# Replace with the actual uploaded file name
file_name = "Output.zip"

with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall("/content")
print("Files extracted successfully!")

Files extracted successfully!


In [ ]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

Torch version: 2.8.0+cu126
CUDA available: True
Device: cuda
GPU name: Tesla T4


Imports & Setup

In [ ]:
import torch
import random
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Trainer, TrainingArguments
from pathlib import Path
import evaluate
import os

Loading data

In [ ]:
import zipfile
import pandas as pd
from pathlib import Path

#  Step 1: Define paths and unzip the file

zip_path = Path("/content/Output.zip")     # uploaded zip file path
extract_dir = Path("/content/Output")      # folder to extract files

# Extract safely
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Extraction complete!")


#  Step 2: Find all CSV files inside folder

csv_files = list(extract_dir.rglob("*.csv"))
if not csv_files:
    raise FileNotFoundError(" No CSV files found in extracted folder.")
print(" CSV files found:", [f.name for f in csv_files])

#  Step 3: Helper function to load CSVs by name

def load_csv(name_pattern):
    for f in csv_files:
        if name_pattern.lower() in f.name.lower():
            print(f" Loading: {f.name}")
            return pd.read_csv(f)
    raise FileNotFoundError(f" Could not find file with '{name_pattern}' in name.")

# Load datasets
train_en = load_csv("train_en")
val_en   = load_csv("val_en")
test_en  = load_csv("test_en")


#  Step 4: Show dataset shapes and previews

print("\n Data loaded successfully!")
print(f"Train shape: {train_en.shape}")
print(f"Validation shape: {val_en.shape}")
print(f"Test shape: {test_en.shape}")

print("\n Sample of training data:")
display(train_en.head())


✅ Extraction complete!
 CSV files found: ['train_hi.csv', 'val_hi.csv', 'train_te.csv', 'train_en.csv', 'val_en.csv', 'test_hi.csv', 'merged_en (1).csv', 'val_te.csv', 'test_te.csv', 'merged_multilang (1).csv', 'test_en.csv', 'train_hi.csv', 'val_hi.csv', 'train_te.csv', 'train_en.csv', 'val_en.csv', 'test_hi.csv', 'merged_en (1).csv', 'val_te.csv', 'test_te.csv', 'merged_multilang (1).csv', 'test_en.csv']
 Loading: train_en.csv
 Loading: val_en.csv
 Loading: test_en.csv

 Data loaded successfully!
Train shape: (1140, 8)
Validation shape: (229, 8)
Test shape: (153, 8)

 Sample of training data:


,id,source,type,prompt,response,lang,orig_id,fp
0,dlg_cbb0155f815386275f372989ec609d9f,train.csv,dialogue,Is this something I should be worried about? S...,The answer depends on how the other areas of y...,en,NaN,cbb0155f815386275f372989ec609d9f
1,dlg_c7613dbed473f5ba3dd4199dc32fa2ee,train.csv,dialogue,I have an eating disorder of binging. I've had...,It can be really frustrating to feel like your...,en,NaN,c7613dbed473f5ba3dd4199dc32fa2ee
2,dlg_a1be9cdc599d68a8c56c03478da9565b,train.csv,dialogue,There's this boy who asked me out awhile ago. ...,Since you've decided to be with the boy who al...,en,NaN,a1be9cdc599d68a8c56c03478da9565b
3,dlg_017a5073f7e494c8490f955fc5546d82,train.csv,dialogue,"Any time things get heated, the past gets brou...",Have you tried to talk about the subjects of t...,en,NaN,017a5073f7e494c8490f955fc5546d82
4,dlg_28599e1f110e9bf0d4a92453fb0058ac,train.csv,dialogue,I am a really shy person. I'm currently in a g...,"A good way to start is the language or ""self t...",en,NaN,28599e1f110e9bf0d4a92453fb0058ac


In [ ]:
import pandas as pd

# Load your dataset
train_en = pd.read_csv("/content/Output/train_en.csv")
val_en = pd.read_csv("/content/Output/val_en.csv")

# Inspect column names
print("Training columns:", train_en.columns.tolist())
print("Validation columns:", val_en.columns.tolist())

# See a few examples
print("\nSample training rows:")
print(train_en.head(3))


Training columns: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp']
Validation columns: ['id', 'source', 'type', 'prompt', 'response', 'lang', 'orig_id', 'fp']

Sample training rows:
                                     id     source      type  \
0  dlg_cbb0155f815386275f372989ec609d9f  train.csv  dialogue   
1  dlg_c7613dbed473f5ba3dd4199dc32fa2ee  train.csv  dialogue   
2  dlg_a1be9cdc599d68a8c56c03478da9565b  train.csv  dialogue   

                                              prompt  \
0  Is this something I should be worried about? S...   
1  I have an eating disorder of binging. I've had...   
2  There's this boy who asked me out awhile ago. ...   

                                            response lang  orig_id  \
0  The answer depends on how the other areas of y...   en      NaN   
1  It can be really frustrating to feel like your...   en      NaN   
2  Since you've decided to be with the boy who al...   en      NaN   

                                

Data preprocessing

In [ ]:
from datasets import Dataset
from transformers import MT5Tokenizer


# Initialize tokenizer
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)

# If your CSVs already have columns like 'input_text' and 'target_text', adjust here
def preprocess_function(examples):
    inputs = examples["prompt"]
    targets = examples["response"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

#  Rename for consistency
train_df = train_en.copy()
val_df   = val_en.copy()
test_df  = test_en.copy()

#  Convert pandas → Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset   = Dataset.from_pandas(val_df)
test_dataset  = Dataset.from_pandas(test_df)

# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val   = val_dataset.map(preprocess_function, batched=True)
tokenized_test  = test_dataset.map(preprocess_function, batched=True)

print("✅ Tokenization complete!")
print("Train samples:", len(tokenized_train))
print("Val samples:", len(tokenized_val))
print("Test samples:", len(tokenized_test))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

✅ Tokenization complete!
Train samples: 1140
Val samples: 229
Test samples: 153


In [ ]:
import re


# 1. CREATE PAIRS LIST FROM DATAFRAME

pairs = list(zip(train_en["prompt"], train_en["response"]))

# 2. TOKEN COLLECTION

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()

for line in pairs:
    input_doc, target_doc = line[0], line[1]

    input_docs.append(input_doc)

    # split words from punctuation
    target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))

    # add START and END tokens
    target_doc = "<START> " + target_doc + " <END>"
    target_docs.append(target_doc)

    # collect input tokens
    for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
        input_tokens.add(token)

    # collect target tokens
    for token in target_doc.split():
        target_tokens.add(token)

# 3. SORT TOKENS

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

print("Input sentences:", len(input_docs))
print("Target sentences:", len(target_docs))
print("Input vocab size:", num_encoder_tokens)
print("Target vocab size:", num_decoder_tokens)


Input sentences: 1140
Target sentences: 1140
Input vocab size: 2945
Target vocab size: 6667


In [ ]:
input_docs

['Is this something I should be worried about? Should I do something about it?',
 "I have an eating disorder of binging. I've had gastric sleeve surgery. I need help with issues of abuse as a child, addiction, and abusive men. I have been in therapy for five months and get no feedback from my therapist.",
 "There's this boy who asked me out awhile ago. I said no because my friends and family would think weirdly of me. How do I be with him and tell him and tell my friends?",
 "Any time things get heated, the past gets brought up. I raise my voice at her to try to get a point across. She asks open-ended questions but always has to turn my own answer against me. She's seven months pregnant. I have drank in the past, but I'm not dependent on alcohol—I only drink at social events.",
 "I am a really shy person. I'm currently in a graduate program which requires lots of presentations, but they always make me feel stressed out and less confident. How can I get more confident?",
 "I'm dealing w

In [ ]:

target_docs

["<START> The answer depends on how the other areas of your son's life are doing . Is he happy or does he seem happy , playing alone during recess ? Does he have friends in other social circles besides the students whom he's with at recess ? How is his academic progress ? How is his social integration among his classmates ? Are there any special or unusual circumstances in the home and family environment ? Go through this list to form a fuller idea of whether your son simply likes alone time and takes this option during recess , or if any if the above areas show stress or difficulty for him and which need to be further understood and handled . <END>",
 "<START> It can be really frustrating to feel like your counselor is not providing you with the help you need . My recommendation in a situation like this would be to let your counselor know how you feel . Specifically tell Your counselor that you don't seem to be getting the amount of feedback you would like . If this doesn't help , the

In [ ]:
input_features_dict = dict([(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict([(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict((i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict((i, token) for token, i in target_features_dict.items())

In [ ]:
input_features_dict

{'!': 0,
 '"': 1,
 '$': 2,
 '%': 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '/': 9,
 '0': 10,
 '00': 11,
 '1': 12,
 '10': 13,
 '100': 14,
 '11': 15,
 '11pm': 16,
 '12': 17,
 '13': 18,
 '14': 19,
 '15': 20,
 '17': 21,
 '18': 22,
 '19': 23,
 '2': 24,
 '20': 25,
 '2014': 26,
 '20s': 27,
 '24': 28,
 '25': 29,
 '3': 30,
 '30': 31,
 '30s': 32,
 '35': 33,
 '4': 34,
 '40': 35,
 '40s': 36,
 '45': 37,
 '48': 38,
 '4th': 39,
 '5': 40,
 '50': 41,
 '50s': 42,
 '5mg': 43,
 '6': 44,
 '60s': 45,
 '7': 46,
 '8': 47,
 '9': 48,
 ':': 49,
 ';': 50,
 '?': 51,
 'A': 52,
 'ADHD': 53,
 'About': 54,
 'Act': 55,
 'Advice': 56,
 'After': 57,
 'All': 58,
 'Alll': 59,
 'Also': 60,
 "Alzheimer's": 61,
 'Am': 62,
 'Amsterdam': 63,
 'An': 64,
 'And': 65,
 'Another': 66,
 'Any': 67,
 'Anytime': 68,
 'Apparently': 69,
 'April': 70,
 'Are': 71,
 'As': 72,
 "Asperger's": 73,
 'At': 74,
 'Baker': 75,
 'Barbie': 76,
 'Basically': 77,
 'Because': 78,
 'Before': 79,
 'But': 80,
 'By': 81,
 'Can': 82,
 'Cheating': 83,
 

In [ ]:
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.

    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [ ]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:

decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
dimensionality = 256 # Dimensionality
batch_size = 10   # The batch size and number of epochs
epochs = 500

#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs) # Compiling

In [ ]:
training_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ 2945)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ 6667)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │  3,278,848 │ input_layer[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, None,     │  7,090,176 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None,      │  1,713,419 │ lstm_1[0][0]      │
│                     │ 6667)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,082,443 (46.09 MB)

 Trainable params: 12,082,443 (46.09 MB)

 Non-trainable params: 0 (0.00 B)

Training

In [ ]:
from transformers import MT5ForConditionalGeneration, TrainingArguments, Trainer
import os

# Disable W&B
os.environ["WANDB_DISABLED"] = "true"

# Load model
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

# Training arguments
training_args = TrainingArguments(
    output_dir="./mt5-finetuned-english",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    eval_strategy="epoch",     # changed from evaluation_strategy
    learning_rate=3e-4
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

# Start training
trainer.train()


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,3.709800,3.024753
2,3.483100,2.879941
3,3.221000,2.839325


TrainOutput(global_step=855, training_loss=4.881297632407026, metrics={'train_runtime': 414.2244, 'train_samples_per_second': 8.256, 'train_steps_per_second': 2.064, 'total_flos': 452080985702400.0, 'train_loss': 4.881297632407026, 'epoch': 3.0})

In [ ]:
trainer.save_model("./mt5-finetuned-english-final")
tokenizer.save_pretrained("./mt5-finetuned-english-final")


('./mt5-finetuned-english-final/tokenizer_config.json',
 './mt5-finetuned-english-final/special_tokens_map.json',
 './mt5-finetuned-english-final/spiece.model',
 './mt5-finetuned-english-final/added_tokens.json')

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

model_name = "./mt5-finetuned-english-final"
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = MT5Tokenizer.from_pretrained(model_name)


In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

# Load your fine-tuned model
model_name = "./mt5-finetuned-english-final"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def chat_with_bot(prompt):
    # Tokenize input
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        padding="max_length"
    ).to(device)

    # Generate response
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=5,
        do_sample=True,             # enable sampling for more variety
        temperature=0.8,            # controls creativity
        top_p=0.9,                  # nucleus sampling
        repetition_penalty=1.8,     # penalizes repetition
        no_repeat_ngram_size=3,     # prevents repeating phrases
        early_stopping=True
    )

    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.strip()

In [ ]:
print("🧠 Prompt: What are the symptoms of depression?")
print("💬 Response:", chat_with_bot("What are the symptoms of depression?"))

🧠 Prompt: What are the symptoms of depression?
💬 Response: It sounds like you are experiencing the symptoms of depression. There are many different types of depression in your life.


In [ ]:
# 🧠 Example Test
print("You:", "How can I calm myself before an exam?")
print("Bot:", chat_with_bot("How can I calm myself before an exam?"))

You: How can I calm myself before an exam?
Bot: It sounds like you're able to talk about your feelings of anxiety. I would suggest that you have a lot of confidence in your life.


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataset (update the path)
df = pd.read_csv("/content/Output/Output/train_en.csv")

# Ensure we have the right columns
df = df[["prompt", "response"]].dropna()

# Build TF-IDF embeddings for all prompts
vectorizer = TfidfVectorizer(stop_words="english")
prompt_vectors = vectorizer.fit_transform(df["prompt"])

def chatbot_reply(user_input):
    # Convert the input into vector form
    user_vec = vectorizer.transform([user_input])

    # Compute similarity with all dataset prompts
    sims = cosine_similarity(user_vec, prompt_vectors)

    # Get the most similar response
    idx = sims.argmax()
    return df.iloc[idx]["response"]

# 💬 Try a few examples
print("🧠 Prompt: Is this something I should be worried about? Should I do something about it?")
print("💬 Response:", chatbot_reply("Is this something I should be worried about? Should I do something about it?"))



🧠 Prompt: Is this something I should be worried about? Should I do something about it?
💬 Response: The answer depends on how the other areas of your son's life are doing.Is he happy or does he seem happy, playing alone during recess?Does he have friends in other social circles besides the students whom he's with at recess?How is his academic progress?How is his social integration among his classmates?Are there any special or unusual circumstances in the home and family environment?Go through this list to form a fuller idea of whether your son simply likes alone time and takes this option during recess, or if any if the above areas show stress or difficulty for him and which need to be further understood and handled.


In [ ]:
# 💬 Try a few examples
print("🧠 Prompt: I have an eating disorder of binging. I've had gastric sleeve surgery. I need help with issues of abuse as a child, addiction, and abusive men. I have been in therapy for five months and get no feedback from my therapist.")
print("💬 Response:", chatbot_reply("I have an eating disorder of binging. I've had gastric sleeve surgery. I need help with issues of abuse as a child, addiction, and abusive men. I have been in therapy for five months and get no feedback from my therapist."))


🧠 Prompt: I have an eating disorder of binging. I've had gastric sleeve surgery. I need help with issues of abuse as a child, addiction, and abusive men. I have been in therapy for five months and get no feedback from my therapist.
💬 Response: It can be really frustrating to feel like your counselor is not providing you with the help you need.My recommendation in a situation like this would be to let your counselor know how you feel. Specifically tell Your counselor that you don't seem to be getting the amount of feedback you would like.If this doesn't help, then you might want to look for another counselor who will be a better fit for you.


Testing

In [ ]:
import pandas as pd
from pathlib import Path
import zipfile

# === Define your zip path and extract directory ===
zip_path = Path("/content/Output.zip")   # path to your uploaded dataset zip
extract_dir = Path("/content/Output")

# Unzip if not already extracted
if not extract_dir.exists():
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

# === Load test dataset ===
test_en = pd.read_csv(extract_dir / "test_en.csv")
print("✅ Test data loaded:", test_en.shape)
print(test_en.head())


✅ Test data loaded: (153, 8)
                                     id     source      type  \
0  dlg_67528adc82beaf3e655f94742ee41082  train.csv  dialogue   
1  dlg_15f7fd00107bf3ed7b36eb982c91c9f7  train.csv  dialogue   
2  dlg_3bfd063fdf1fdd6497ce3e700153955e  train.csv  dialogue   
3  dlg_aca61c044fd62e387d203d3bf10c7b67  train.csv  dialogue   
4  dlg_0e52e7de87752562ec8eca45c4f7e5c6  train.csv  dialogue   

                                              prompt  \
0  My boyfriend of five months expresses how much...   
1  I'm a teenager, and I created a sort of imagin...   
2  How do you know you have the right therapist f...   
3  I have dog obsession disorder and I am having ...   
4  My fiancé and I have been together for 3 years...   

                                            response lang  orig_id  \
0  New York, New York...it's not your boyfriend's...   en      NaN   
1  Hi, I'm Amelia! You know, it does sounds like ...   en      NaN   
2  Thinking whether or not you have the

In [ ]:
!pip install nltk rouge-score

import nltk
nltk.download("punkt")

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from tqdm import tqdm

smooth = SmoothingFunction().method1

preds, refs = [], []

# -----------------------------
# Generate predictions
# -----------------------------
for i in tqdm(range(len(test_en))):
    inp = test_en["prompt"][i]
    ref = test_en["response"][i]

    pred = generate_response(inp)  # your model's inference function

    preds.append(pred)
    refs.append(ref)

# -----------------------------
# BLEU SCORE
# -----------------------------
bleu_scores = []
for p, r in zip(preds, refs):
    bleu = sentence_bleu(
        [r.split()],
        p.split(),
        smoothing_function=smooth
    )
    bleu_scores.append(bleu)

avg_bleu = sum(bleu_scores) / len(bleu_scores)

# -----------------------------
# ROUGE SCORE
# -----------------------------
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge1_scores = []
rougeL_scores = []

for p, r in zip(preds, refs):
    scores = scorer.score(r, p)   # correct order: reference, prediction
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

# -----------------------------
# Output
# -----------------------------
print("✅ BLEU Score:", avg_bleu)
print("✅ ROUGE-1:", avg_rouge1)
print("✅ ROUGE-L:", avg_rougeL)


100%|██████████| 153/153 [03:28<00:00,  1.36s/it]
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram 

✅ BLEU: 0.0009406853683688286
✅ ROUGE-1: 0.21660545034815407
✅ ROUGE-L: 0.13543522381653492


In [ ]:
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
import torch

model_name = "/content/mt5-finetuned-english-final"  # or your saved folder path
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    output = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="/content/mt5-finetuned-english-final",
    do_train=True,
    do_eval=True,
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

save_path = "./mt5_mental_health_chatbot"

# ✅ Save model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ Model and tokenizer saved successfully!")

# ✅ Reload anytime later
model = MT5ForConditionalGeneration.from_pretrained(save_path)
tokenizer = T5Tokenizer.from_pretrained(save_path)


✅ Model and tokenizer saved successfully!
